Used libraries:

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import ElasticNet
import optuna

## Preprocessing

In [133]:
# Import dataset
train_df = pd.read_csv('../../backend/data/training-housing-dataset.csv')

In [132]:
train_df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus,total_rooms,area_per_room,amenity_count
0,1.000000,0.553672,4,2,2,1,0,0,0,1,1,1,1,6,0.243257,2
1,0.990741,0.209040,4,2,2,1,0,0,1,0,2,0,2,6,0.083530,2
2,0.990741,0.694915,3,2,2,1,0,0,0,0,0,1,1,5,0.390416,1
3,0.972222,0.491525,4,1,2,1,0,1,0,0,2,0,1,5,0.277298,2
4,0.972222,0.559322,4,2,2,1,1,1,0,1,1,1,0,6,0.245876,4


In [134]:
# Scaling Data
scaled_columns = ["price", "area", "area_per_room"]
scaler = MinMaxScaler()
train_df[scaled_columns] = scaler.fit_transform(df[scaled_columns])

In [135]:
# Creating training and testing datasets
X = train_df.drop(columns="price")
Y = train_df['price']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15, random_state=42)

In [ ]:
# Creating objective function for model optimization

def objective_elastic(trial):
    alpha = trial.suggest_loguniform('alpha', 1e-4, 1e1)
    L1_ratio = trial.suggest_loguniform('l1_ratio', 0.0, 1.0)
    
    # Creating Elastic Net model with suggested hyperparams
    model = ElasticNet(alpha=alpha, l1_ratio=L1_ratio, random_state=42)
    